In [ ]:
# ===============================
# INSTALL (run once)
# ===============================
# !pip install -U gradio langgraph langchain langchain-openai

import gradio as gr
from typing import TypedDict, Optional, List, Dict
from langgraph.graph import StateGraph, END
from langgraph.types import interrupt
from langchain_openai import ChatOpenAI

# ===============================
# GLOBAL FEEDBACK STORE
# ===============================
FEEDBACK_STORE: List[Dict] = []

# ===============================
# STATE
# ===============================
class QAState(TypedDict):
    question: str
    answer: Optional[str]
    feedbackRating: Optional[int]
    feedbackComment: Optional[str]

# ===============================
# LLM
# ===============================
llm = ChatOpenAI(
    model="gpt-4o-mini",
    temperature=0
)

# ===============================
# LANGGRAPH NODES
# ===============================

def answer_question(state: QAState) -> QAState:
    prompt = f'''
    Answer Clearly: 

    "{state['question']}"
    '''

    print("Prompt to LLM:", prompt)
    response = llm.invoke([{"role": "user", "content": prompt}])

    print("LLM Response:", response)
    state["answer"] = response.content
    return state


def feedback_interrupt(state: QAState) -> QAState:
    print("Checking for feedback...")
    if state.get("feedbackRating") is None:
        print("Requesting feedback from user.")
        interrupt({
            "type": "feedback_request",
            "answer": state["answer"]
        })
        print("Feedback received:", state.get("feedbackRating"), state.get("feedbackComment"))
    return state


def store_feedback(state: QAState) -> QAState:
    print("Storing feedback...")
    FEEDBACK_STORE.append({
        "question": state["question"],
        "answer": state["answer"],
        "rating": state["feedbackRating"],
        "comment": state["feedbackComment"]
    })
    print("Current Feedback Store:", FEEDBACK_STORE)
    return state

# ===============================
# GRAPH
# ===============================
builder = StateGraph(QAState)

builder.add_node("answer", answer_question)
builder.add_node("feedback_interrupt", feedback_interrupt)
builder.add_node("store_feedback", store_feedback)

builder.set_entry_point("answer")
builder.add_edge("answer", "feedback_interrupt")
builder.add_edge("feedback_interrupt", "store_feedback")
builder.add_edge("store_feedback", END)

graph = builder.compile()

# ===============================
# GRADIO LOGIC
# ===============================

session_state = {}

def chat_fn(user_input, history):
    state = {
        "question": user_input,
        "answer": None,
        "feedbackRating": None,
        "feedbackComment": None
    }

    result = graph.invoke(state)
    session_state["state"] = result

    history.append({"role": "user", "content": user_input})
    history.append({"role": "assistant", "content": result["answer"]})

    return history, gr.update(visible=True)


def submit_feedback(rating, comment, history):
    state = session_state["state"]
    state["feedbackRating"] = rating
    state["feedbackComment"] = comment

    graph.invoke(state)

    history.append({
        "role": "assistant",
        "content": f"📝 Thanks for the feedback! ⭐ {rating}/5"
    })

    return history, gr.update(visible=False), None, ""

# ===============================
# GRADIO UI
# ===============================
with gr.Blocks() as demo:
    gr.Markdown("## 🤖 Q&A Chatbot with Feedback Loop")

    # chatbot = gr.Chatbot()
    chatbot = gr.Chatbot()
    user_input = gr.Textbox(placeholder="Ask a question...")
    send_btn = gr.Button("Send")

    with gr.Group(visible=False) as feedback_box:
        gr.Markdown("### ⭐ Rate the answer")
        rating = gr.Slider(1, 5, step=1, label="Rating")
        comment = gr.Textbox(label="Optional comment")
        submit_btn = gr.Button("Submit Feedback")

    send_btn.click(
        chat_fn,
        inputs=[user_input, chatbot],
        outputs=[chatbot, feedback_box]
    )

    submit_btn.click(
        submit_feedback,
        inputs=[rating, comment, chatbot],
        outputs=[chatbot, feedback_box, rating, comment]
    )

demo.launch()


* Running on local URL:  http://127.0.0.1:7865
* To create a public link, set `share=True` in `launch()`.


Prompt to LLM: 
    Answer Clearly: 

    "hi"
    
LLM Response: content='Hello! How can I assist you today?' additional_kwargs={'refusal': None} response_metadata={'token_usage': {'completion_tokens': 9, 'prompt_tokens': 18, 'total_tokens': 27, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_provider': 'openai', 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_c4585b5b9c', 'id': 'chatcmpl-Cyca2QAt8SY51nai82M9YKE445FIL', 'service_tier': 'default', 'finish_reason': 'stop', 'logprobs': None} id='lc_run--019bc6a1-4843-7662-929f-783819337123-0' tool_calls=[] invalid_tool_calls=[] usage_metadata={'input_tokens': 18, 'output_tokens': 9, 'total_tokens': 27, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}}
Checking for feedback...
Requesting feedback from 